

```
# This is formatted as code
```

# **Installing Libraries**

In [ ]:
!pip install torch torchvision
!pip install ultralytics
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

# **Importing Libraries & Downloading Dataset**

In [ ]:
import os
import torch
import torchvision
from ultralytics import YOLO
import logging
from datetime import datetime
import zipfile
from roboflow import Roboflow
rf = Roboflow(api_key="QXLoOvEefX5saK5iVqKR")
project = rf.workspace("cucumber-bdhva").project("my-first-project-q8blq")
version = project.version(1)
dataset = version.download("yolov8")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to My-First-Project-1 in yolov8:: 100%|██████████| 19688/19688 [00:07<00:00, 2654.29it/s]


# **Logging Configuration**

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'training_log_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# **Path Configuration & NMS GPU Compatibility Workaround**

In [ ]:
class Config:
    DATASET_YAML = "/content/My-First-Project-1/data.yaml"
    TEST_IMAGE = "/content/My-First-Project-1/valid/images/Anthracnose-143-_jpg.rf.10ea4f0ea2f9d46bfed5c7a015616204.jpg"
    OUTPUT_DIR = "/content/"

original_nms = torchvision.ops.nms

def nms_gpu_workaround(boxes, scores, iou_threshold):
    try:
        if boxes.is_cuda:
            boxes_cpu = boxes.cpu()
            scores_cpu = scores.cpu()
            keep = original_nms(boxes_cpu, scores_cpu, iou_threshold)
            return keep.to(boxes.device)
        return original_nms(boxes, scores, iou_threshold)
    except Exception as e:
        logger.error(f"NMS computation failed: {str(e)}")
        raise

torchvision.ops.nms = nms_gpu_workaround

# **YOLOv8 Model Training and Inference Pipeline**

In [ ]:
class YOLOTrainer:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.output_dir = Config.OUTPUT_DIR
        os.makedirs(self.output_dir, exist_ok=True)
        self.model = None

    def train_model(self):
        try:
            logger.info("Starting model training...")
            self.model = YOLO('yolov8n.pt')
            self.model.train(
                data=Config.DATASET_YAML,
                epochs=60,
                imgsz=640,
                batch=16,
                name='yolov8_train_opt',
                project=self.output_dir,
                device=self.device,
                patience=20,
                verbose=True,
                lr0=0.001,
                lrf=0.01,
                warmup_epochs=3,
                workers=16,
                weight_decay=0.0005,
                momentum=0.937,
                box=7.5,
                cls=0.5,
                dfl=1.5,
                optimizer='SGD',
                augment=False
            )
            logger.info("Training completed successfully")
            return self.model
        except Exception as e:
            logger.error(f"Training failed: {str(e)}")
            raise

    def evaluate_model(self):
        if not self.model:
            raise ValueError("Model not trained yet!")
        try:
            logger.info("Evaluating model...")
            metrics = self.model.val()
            eval_metrics = {
                'mAP@0.5': metrics.box.map50,
                'mAP@0.5:0.95': metrics.box.map,
                'Precision': metrics.box.p[0],
                'Recall': metrics.box.r[0]
            }
            for metric, value in eval_metrics.items():
                logger.info(f"{metric}: {value:.3f}")
            return eval_metrics
        except Exception as e:
            logger.error(f"Evaluation failed: {str(e)}")
            raise

    def run_inference(self, image_path, confidence_threshold=0.65):
        if not self.model:
            raise ValueError("Model not trained yet!")
        try:
            logger.info(f"Running inference on {image_path}")
            results = self.model.predict(
                source=image_path,
                conf=confidence_threshold,
                iou=0.6,
                save=True,
                project=self.output_dir,
                name='inference_opt',
                save_txt=True,
                save_conf=True
            )
            for result in results:
                logger.info("Detected Objects:")
                if result.boxes:
                    for box in result.boxes:
                        cls = int(box.cls)
                        conf = float(box.conf)
                        bbox = box.xyxy[0].tolist()
                        logger.info(f"Class: {self.model.names[cls]}, Confidence: {conf:.2f}, BBox: {bbox}")
                else:
                    logger.info("No objects detected")
        except Exception as e:
            logger.error(f"Inference failed: {str(e)}")
            raise

    def export_model(self, format='onnx'):
        if not self.model:
            raise ValueError("Model not trained yet!")
        try:
            logger.info(f"Exporting model to {format}...")
            export_path = self.model.export(
                format=format,
                imgsz=640,
                optimize=True if format == 'onnx' else False
            )
            logger.info(f"Model exported successfully to {export_path}")
        except Exception as e:
            logger.error(f"Export failed: {str(e)}")
            raise

def main():
    try:
        trainer = YOLOTrainer()
        logger.info("Starting training process...")
        trainer.train_model()
        logger.info("Starting evaluation process...")
        trainer.evaluate_model()
        logger.info("Starting inference process...")
        trainer.run_inference(Config.TEST_IMAGE)
        logger.info("Starting export process...")
        trainer.export_model(format='onnx')
        logger.info("All processes completed successfully")
    except Exception as e:
        logger.error(f"Program failed: {str(e)}")
        raise

if __name__ == "__main__":
    main()

100%|██████████| 6.25M/6.25M [00:00<00:00, 135MB/s]


engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/My-First-Project-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_train_opt, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/content/, rect=False, resume=Fals

100%|██████████| 755k/755k [00:00<00:00, 20.7MB/s]

Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 22        [15, 18, 21]  1    752872  ultralytics.nn.modules.head.Detect           [8, [64, 128, 256]]           
Model summary: 129 layers, 3,012,408 parameters, 3,012,392 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

                   all        240        310      0.937      0.904      0.938      0.722
 Cucumber__Anthracnose         30         30      0.953      0.933      0.976       0.69
  Cucumber__Belly_Rote         30         65      0.886      0.892      0.866      0.425
Cucumber__Downy_mildew         30         31      0.931      0.968      0.967      0.851
Cucumber__Gummy_Stem_Blight         30         35      0.967      0.943      0.972      0.646
Cucumber__Healthy_Fruit         30         47      0.916      0.695       0.85      0.629
Cucumber__Healthy_Leaf         30         30      0.986      0.967      0.993      0.903
Cucumber__Powdery_Mildew         30         30      0.952          1      0.995      0.967
Cucumber__Pythium_Fruit_Rot         30         42      0.904      0.833       0.89      0.666
Speed: 0.5ms preprocess, 2.5ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to /content/yolov8_train_opt
Ultralytics 8.3.144 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0

val: Scanning /content/My-First-Project-1/valid/labels.cache... 240 images, 0 backgrounds, 0 corrupt: 100%|██████████| 240/240 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 20, len(boxes) = 310. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.70it/s]


                   all        240        310      0.937      0.904      0.939      0.723
 Cucumber__Anthracnose         30         30      0.953      0.933      0.976      0.691
  Cucumber__Belly_Rote         30         65      0.886      0.892      0.866      0.427
Cucumber__Downy_mildew         30         31      0.931      0.968      0.967      0.851
Cucumber__Gummy_Stem_Blight         30         35      0.967      0.943      0.971      0.647
Cucumber__Healthy_Fruit         30         47      0.916      0.695       0.85      0.629
Cucumber__Healthy_Leaf         30         30      0.986      0.967      0.993      0.903
Cucumber__Powdery_Mildew         30         30      0.952          1      0.995      0.969
Cucumber__Pythium_Fruit_Rot         30         42      0.904      0.833       0.89      0.667
Speed: 1.8ms preprocess, 5.4ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to /content/yolov8_train_opt2

image 1/1 /content/My-First-Project-1/valid/images/Anthracn

# **Results Folder Compression to ZIP Archive**

In [ ]:
def zip_folder(folder_path, output_zip):
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as z:
        for root, _, files in os.walk(folder_path):
            for f in files:
                z.write(os.path.join(root, f),
                       os.path.relpath(os.path.join(root, f), folder_path))

zip_folder('/content/yolov8_train_opt2', '/content/yolov8_ِِAABB_train_opt2.zip')
zip_folder('/content/yolov8_train_opt', '/content/yolov8_AABB_train_opt.zip')